# Develop process


In [2]:
import neqsim
from neqsim.thermo.thermoTools import *
import pandas as pd

# Define input parameters

In [62]:
inputdata = {
  'feedFlowRateTrain1': 11411.9,
  'feedPressure': 89.0,
  'feedTemperature': 5.0,
  'sep1Pressure': 85.0,
  'cooler1T': -5.0,
  'expOutPressure': 67.0,
  'glycolFlow': 1.0,
  
}

# Define fluid for feed

In [51]:
feedFluid = {'ComponentName':  ['water', 'MEG', "methane", "ethane", "C6", "C7"], 
        'MolarComposition[-]':  [1.0, 0.0, 0.5, 0.1,0.1, 0.3], 
        'MolarMass[kg/mol]': [None,None, None,None, 0.091, 0.19],
        'RelativeDensity[-]': [None,None,None,None, 0.7, 0.86 ]
  }

reservoirFluiddf = pd.DataFrame(feedFluid) 
print(reservoirFluiddf.head(20).to_string()) 
fluid7 = fluid_df(reservoirFluiddf)

glycolFluid = fluid7.clone()
glycolFluid.setMolarComposition([0.0, 1.0, 0.0, 0.0,0.0, 0.0])

  ComponentName  MolarComposition[-]  MolarMass[kg/mol]  RelativeDensity[-]
0         water                  1.0                NaN                 NaN
1           MEG                  0.0                NaN                 NaN
2       methane                  0.5                NaN                 NaN
3        ethane                  0.1                NaN                 NaN
4            C6                  0.1              0.091                0.70
5            C7                  0.3              0.190                0.86


# Creating Kolsnes dew point process

In [63]:
from neqsim.process import expander, mixer, stream, cooler, valve, separator3phase,clearProcess,runProcess

clearProcess()
feedStream = stream(fluid7)
feedStream.setFlowRate(inputdata['feedFlowRateTrain1'], 'kg/hr')
feedStream.setPressure(inputdata['feedPressure'], 'barg')
feedStream.setTemperature(inputdata['feedTemperature'], 'C')

glycolFeedStream = stream(glycolFluid)
glycolFeedStream.setFlowRate(inputdata['glycolFlow'], 'kg/hr')
glycolFeedStream.setTemperature(inputdata['feedTemperature'], 'C')
glycolFeedStream.setPressure(inputdata['feedPressure'], 'barg')

slugCatcher = separator3phase(feedStream)

gasFromSlugCatcher = stream(slugCatcher.getGasOutStream())

valve1 = valve(gasFromSlugCatcher)
valve1.setOutletPressure(inputdata['sep1Pressure'])

sep1 = separator3phase(valve1.getOutStream())

cooler1 = cooler(sep1.getGasOutStream())
cooler1.setOutTemperature(inputdata['cooler1T'], 'C')

sep2 = separator3phase(cooler1.getOutStream())

mixer1 = mixer()
mixer1.addStream(sep2.getGasOutStream())
mixer1.addStream(glycolFeedStream)

expander1 = expander(mixer1.getOutStream(), inputdata['expOutPressure'])

sep3 = separator3phase(expander1.getOutStream())

gasToExport = stream(sep3.getGasOutStream())

runProcess()




In [64]:
printFrame(gasToExport.getFluid())
print('temperatutre out expander ', expander1.getOutStream().getTemperature('C'))

gasToExport.setPressure(70.0, 'bara')
print('hydrate/water dew point temperature ', (gasToExport.getHydrateEquilibriumTemperature()-273.15))

                        total               gas                   
             water  4.4647E−6         4.4647E−6    [mole fraction]
               MEG 1.22401E−7        1.22401E−7    [mole fraction]
           methane 9.28392E−1        9.28392E−1    [mole fraction]
            ethane 7.09026E−2        7.09026E−2    [mole fraction]
             C6_PC 7.00429E−4        7.00429E−4    [mole fraction]
             C7_PC 2.95263E−8        2.95263E−8    [mole fraction]
                                                                  
           Density                    7.30518E1           [kg/m^3]
     PhaseFraction                          1E0    [mole fraction]
         MolarMass  1.70901E1         1.70901E1          [kg/kmol]
          Z factor                   7.50749E−1                [-]
Heat Capacity (Cp)                    3.33542E0          [kJ/kg*K]
Heat Capacity (Cv)                     1.6739E0          [kJ/kg*K]
    Speed of Sound                    3.73632E2            [m/